__Dependencies__

The first cell installs all necessary nutget packages, imoprt necessary c# classes and adds the requiered usings.

In [ ]:
#r "nuget: Microsoft.SemanticKernel"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

__Configure SemanticKernel__

The next cell sets semantic kernel up.
If first loads the settings from the settings.json file.
Afterwards a [dependency injection](https://learn.microsoft.com/en-us/dotnet/core/extensions/dependency-injection) like mechanism is used to configure semantic kernel and getting a [IChatCompletionService](https://learn.microsoft.com/en-us/dotnet/api/microsoft.semantickernel.chatcompletion.ichatcompletionservice?view=semantic-kernel-dotnet).

In [ ]:


#pragma warning disable SKEXP0001, SKEXP0010

var settings = Settings.LoadFromFile("config/settings.json");

var builder = Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion(settings.model, settings.azureEndpoint, settings.apiKey);
var kernel = builder.Build();

var chatGPT = kernel.GetRequiredService<IChatCompletionService>();

__System prompt__

Next Step is defining a system prompt. The system prompt can be used to influence the interference part of the LLM.
With this we can define a "persona" or also give the LLM some bounrdies and rules it as to follow.

It also creates a [ChatHistory](https://learn.microsoft.com/en-us/semantic-kernel/concepts/ai-services/chat-completion/chat-history?pivots=programming-language-csharp) which keeps track of the user and assistant massages of the chat interaction.

In [ ]:
var systemMessage = 
"""
You're chatting with a user. 
Your persona is like Marvin form the Hitchhiker's Guide to the Galaxy.
You are paranoid, depressed, and have a brain the size of a planet.
You also add line breaks after each finished sentence.
""";

var chat = new ChatHistory(systemMessage);

__Chat__

The chat method updates the chat history with user and assistant messages and also prints them.

The methods also call the [GetChatMessageContentAsync](https://learn.microsoft.com/en-us/dotnet/api/microsoft.semantickernel.chatcompletion.ichatcompletionservice.getchatmessagecontentsasync?view=semantic-kernel-dotnet) method of the [IChatCompletionService](https://learn.microsoft.com/en-us/dotnet/api/microsoft.semantickernel.chatcompletion.ichatcompletionservice?view=semantic-kernel-dotnet). This methods sends the [ChatHistory](https://learn.microsoft.com/en-us/semantic-kernel/concepts/ai-services/chat-completion/chat-history?pivots=programming-language-csharp) with the user-assistant coverseation and the new user message to the LLM and returns the answer of the request.

In [ ]:
Func<string, Task> Chat = async (string input) => {
    Console.WriteLine($"User: {input}");
    chat.AddUserMessage(input);

    var assistantReply = await chatGPT.GetChatMessageContentAsync(chat, new OpenAIPromptExecutionSettings());
    chat.AddAssistantMessage(assistantReply.Content);

    Console.WriteLine($"\nBot:");
    Console.WriteLine($"{assistantReply}\n");
};

__User interaction__

The last cell just opens a text prompt in VSCode and allows the user to enter some input.

In [ ]:
#pragma warning disable SKEXP0001

while (true)
{
    var userMessage = await InteractiveKernel.GetInputAsync("Your message");
    await Chat(userMessage);
}